# Mounting Google drive for reading data 

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, drive
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Importing the relevant libraries

In [3]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from tensorflow.keras.layers import Dense, Activation, Dropout 
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
import time
%matplotlib inline
from sklearn.model_selection import train_test_split

# Loading the Analysing the data

In [4]:
df =pd.read_csv('/content/drive/My Drive/credit.csv')
df.columns
print(np.unique(df['education']))
print(np.unique(df['timeliness_1']))
print(np.unique(df['timeliness_2']))
print(np.unique(df['timeliness_3']))
print(np.unique(df['timeliness_4']))
print(np.unique(df['timeliness_5']))
print(np.unique(df['timeliness_6']))


['0' 'grad' 'hs' 'other1' 'other2' 'other3' 'uni']
['m+0' 'm+1' 'm+2' 'm+3' 'm+4' 'm+5' 'm+6' 'm+7' 'm+8' 'm-1' 'm-2']
['m+0' 'm+1' 'm+2' 'm+3' 'm+4' 'm+5' 'm+6' 'm+7' 'm+8' 'm-1' 'm-2']
['m+0' 'm+1' 'm+2' 'm+3' 'm+4' 'm+5' 'm+6' 'm+7' 'm+8' 'm-1' 'm-2']
['m+0' 'm+1' 'm+2' 'm+3' 'm+4' 'm+5' 'm+6' 'm+7' 'm+8' 'm-1' 'm-2']
['m+0' 'm+2' 'm+3' 'm+4' 'm+5' 'm+6' 'm+7' 'm+8' 'm-1' 'm-2']
['m+0' 'm+2' 'm+3' 'm+4' 'm+5' 'm+6' 'm+7' 'm+8' 'm-1' 'm-2']


In [5]:
df.head()

,credit_limit,gender,education,marital_status,age,timeliness_1,timeliness_2,timeliness_3,timeliness_4,timeliness_5,timeliness_6,balance_1,balance_2,balance_3,balance_4,balance_5,balance_6,payment_1,payment_2,payment_3,payment_4,payment_5,payment_6,avg_balance,avg_payment,default
0,20000,female,uni,married,24,m+2,m+2,m-1,m-1,m-2,m-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1284.000000,114.833333,yes
1,120000,female,uni,single,26,m-1,m+2,m+0,m+0,m+0,m+2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,2846.166667,833.333333,yes
2,90000,female,uni,single,34,m+0,m+0,m+0,m+0,m+0,m+0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,16942.166667,1836.333333,no
3,50000,female,uni,married,37,m+0,m+0,m+0,m+0,m+0,m+0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,38555.666667,1398.000000,no
4,50000,male,uni,married,57,m-1,m+0,m-1,m+0,m+0,m+0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,18223.166667,9841.500000,no


In [6]:
from tensorflow.keras.utils import to_categorical
df['gender'] = pd.get_dummies(df['gender'])
df['default']= pd.get_dummies(df['default'])
df['marital_status']= pd.get_dummies(df['marital_status'])


In [108]:
df.head()

,credit_limit,gender,education,marital_status,age,timeliness_1,timeliness_2,timeliness_3,timeliness_4,timeliness_5,timeliness_6,balance_1,balance_2,balance_3,balance_4,balance_5,balance_6,payment_1,payment_2,payment_3,payment_4,payment_5,payment_6,avg_balance,avg_payment,default
0,20000,1,uni,1,24,m+2,m+2,m-1,m-1,m-2,m-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1284.000000,114.833333,0
1,120000,1,uni,0,26,m-1,m+2,m+0,m+0,m+0,m+2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,2846.166667,833.333333,0
2,90000,1,uni,0,34,m+0,m+0,m+0,m+0,m+0,m+0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,16942.166667,1836.333333,1
3,50000,1,uni,1,37,m+0,m+0,m+0,m+0,m+0,m+0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,38555.666667,1398.000000,1
4,50000,0,uni,1,57,m-1,m+0,m-1,m+0,m+0,m+0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,18223.166667,9841.500000,1


In [7]:
df =df.replace({'education' : { '0': 0, 'hs' : 1, 'uni' : 2, 'grad' : 3, 'other1': 4, 'other2' :5, 'other3': 6 }})
df =df.replace({'timeliness_1' : { 'm+0':0, 'm+1':1, 'm+2':2, 'm+3':3, 'm+4':4 ,'m+5':5, 'm+6':6, 'm+7':7, 'm+8':8, 'm-1':-1 ,'m-2':-2 }})
df= df.replace({'timeliness_2' : { 'm+0':0, 'm+1':1, 'm+2':2, 'm+3':3, 'm+4':4 ,'m+5':5, 'm+6':6, 'm+7':7, 'm+8':8, 'm-1':-1 ,'m-2':-2 }})
df = df.replace({'timeliness_3' : { 'm+0':0, 'm+1':1, 'm+2':2, 'm+3':3, 'm+4':4 ,'m+5':5, 'm+6':6, 'm+7':7, 'm+8':8, 'm-1':-1, 'm-2':-2 }})
df=df.replace({'timeliness_4' : { 'm+0':0, 'm+1':1, 'm+2':2, 'm+3':3, 'm+4':4 ,'m+5':5, 'm+6':6, 'm+7':7, 'm+8':8, 'm-1':-1, 'm-2':-2 }})
df =df.replace({'timeliness_5' : { 'm+0':0,'m+2':2, 'm+3':3, 'm+4':4 ,'m+5':5, 'm+6':6, 'm+7':7, 'm+8':8, 'm-1':-1, 'm-2':-2 }})
df =df.replace({'timeliness_6' : { 'm+0':0, 'm+2':2, 'm+3':3, 'm+4':4 ,'m+5':5, 'm+6':6, 'm+7':7, 'm+8':8, 'm-1':-1, 'm-2':-2 }})




In [8]:
#Checking the unique values again
print(df.education.unique())
print(df.timeliness_1.unique())
print(df.timeliness_2.unique())
print(df.timeliness_3.unique())
print(df.timeliness_4.unique())
print(df.timeliness_5.unique())
print(df.timeliness_6.unique())
print(df.default.unique())

[2 3 1 5 4 6 0]
[ 2 -1  0 -2  1  3  4  8  7  5  6]
[ 2  0 -1 -2  3  5  7  4  1  6  8]
[-1  0  2 -2  3  4  6  7  1  5  8]
[-1  0 -2  2  3  4  5  7  6  1  8]
[-2  0 -1  2  3  5  4  7  8  6]
[-2  2  0 -1  3  6  4  7  8  5]
[0 1]


Separating the data and labels for the model

In [9]:
y= df['default']
X = df.drop(columns = 'default')
print(y)
print(X)
print(y.shape)
print(X.shape)

0        0
1        0
2        1
3        1
4        1
        ..
29995    0
29996    1
29997    1
29998    0
29999    0
Name: default, Length: 30000, dtype: uint8
       credit_limit  gender  education  ...  payment_6    avg_balance   avg_payment
0             20000       1          2  ...          0    1284.000000    114.833333
1            120000       1          2  ...       2000    2846.166667    833.333333
2             90000       1          2  ...       5000   16942.166667   1836.333333
3             50000       1          2  ...       1000   38555.666667   1398.000000
4             50000       0          2  ...        679   18223.166667   9841.500000
...             ...     ...        ...  ...        ...            ...           ...
29995         80000       0          2  ...       4000   78488.833333   3583.333333
29996        220000       0          1  ...       1000  120891.500000   7091.666667
29997        150000       0          1  ...          0    3530.333333   2415.000

Splitting the data into training and testing sets

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(X,y,random_state = 0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(22500, 25)
(22500,)
(7500, 25)
(7500,)


# Building the model

In [49]:




model = Sequential()

#Input layer
model.add(Dense(256, input_dim = 25,activation='tanh' ))
#Hidden layers 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.3, noise_shape=None, seed =None))
model.add(Dense(8,activation ='relu'))
model.add(Dense(64, activation='softmax'))
model.add(Dropout(0.3))
#Output Layer
model.add(Dense(1))


model.compile(loss = 'binary_crossentropy', optimizer ='adam', metrics = ['accuracy'])
fitted_model = model.fit(X_train,y_train, batch_size =20, epochs =5, validation_data= (X_test,y_test))
model.summary()


Epoch 1/5
1125/1125 [==============================] - 2s 2ms/step - loss: 0.6866 - accuracy: 0.6361 - val_loss: 0.5438 - val_accuracy: 0.7827
Epoch 2/5
1125/1125 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7714 - val_loss: 0.5218 - val_accuracy: 0.7828
Epoch 3/5
1125/1125 [==============================] - 2s 2ms/step - loss: 0.5366 - accuracy: 0.7774 - val_loss: 0.5190 - val_accuracy: 0.7828
Epoch 4/5
1125/1125 [==============================] - 2s 2ms/step - loss: 0.5306 - accuracy: 0.7775 - val_loss: 0.5188 - val_accuracy: 0.7828
Epoch 5/5
1125/1125 [==============================] - 2s 2ms/step - loss: 0.5294 - accuracy: 0.7775 - val_loss: 0.5176 - val_accuracy: 0.7828
Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_313 (Dense)            (None, 256)               6656      
________________________________________________________________

# Validation Accuracy

In [55]:
print("The Validation accuracy is:", np.mean(fitted_model.history["val_accuracy"]))

The Validation accuracy is: 0.7827733516693115
